In [1]:
#!pip install boto3
import boto3
from configparser import ConfigParser
import io

In [2]:
# Carregar as configurações do arquivo ini
config = ConfigParser()
config.read("../config/config.ini")

['../config/config.ini']

In [3]:
# Configurações para conexão com o MinIO/S3
endpoint = config.get("MinIO", "endpoint")
access_key = config.get("MinIO", "access_key")
secret_key = config.get("MinIO", "secret_key")
bucket_raw = config.get("Bucket", "bucket_raw")
bucket_context = config.get("Bucket", "bucket_context")
prefix_censo_csv = config.get("Bucket", "prefix_censo_csv") # Prefixo do arquivo CSV na raw
prefix_censo_municipios_csv = config.get("Bucket", "prefix_censo_municipios_csv") # Prefixo do arquivo somente com as informações de saúde que vai para a context

# Nome do arquivo a ser lido
source_filename = config.get("RAW_BUCKET_FILES", "censo_municipios_csv")
target_filename = config.get("FILE", "censo_municipios_csv")

In [4]:
print("endpoint:", endpoint)
print("access_key:", access_key)
print("secret_key:", secret_key)
print("bucket_raw:", bucket_raw)
print("bucket_context:", bucket_context)
print("prefix_censo_csv:", prefix_censo_csv)
print("prefix_censo_municipios_csv:", prefix_censo_municipios_csv)
print("source_filename:", source_filename)
print("target_filename:", target_filename)

endpoint: http://minio:9000
access_key: aulafia
secret_key: aulafia@123
bucket_raw: raw
bucket_context: context
prefix_censo_csv: bases_censo_csv/
prefix_censo_municipios_csv: censo_municipios_csv/
source_filename: Atlas 2013_municipal, estadual e Brasil-MUN 91-00-10.csv
target_filename: CensoMunicipios.csv


In [5]:
# Inicializar o cliente boto3 para S3
minio_client = boto3.client("s3", 
                            endpoint_url=endpoint,
                            aws_access_key_id=access_key,
                            aws_secret_access_key=secret_key
)

In [6]:
print(bucket_raw + prefix_censo_csv + source_filename)

rawbases_censo_csv/Atlas 2013_municipal, estadual e Brasil-MUN 91-00-10.csv


In [7]:
# Obter o objeto do arquivo do bucket de origem
response = minio_client.get_object(Bucket=bucket_raw,
                                   Key=prefix_censo_csv + 
                                   source_filename)
csv_stream = response['Body']

In [8]:
# Iniciar um fluxo de gravação em buffer para o arquivo de destino
output_buffer = io.BytesIO()

In [9]:
csv_stream

In [10]:
# Ler e escrever em partes para lidar com arquivos grandes
chunk_size = 1024 * 1024  # Tamanho de cada parte em bytes (1 MB)

while True:
    chunk = csv_stream.read(chunk_size)
    if not chunk:
        break
    output_buffer.write(chunk)

In [11]:
# Decodificar o conteúdo do CSV usando UTF-8
decoded_csv_content = output_buffer.getvalue().decode('utf-8')

In [12]:
# Codificar novamente o conteúdo em UTF-8
encoded_csv_content = decoded_csv_content.encode('utf-8')

In [13]:
# Enviar o arquivo codificado para o bucket de destino
minio_client.put_object(Bucket=bucket_context,
                        Key=target_filename,
                        Body=encoded_csv_content)

{'ResponseMetadata': {'RequestId': '177F5EA93EAE9863',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"0b825fd116866b19bc14e190b9683b9c"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '177F5EA93EAE9863',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block',
   'date': 'Sun, 27 Aug 2023 22:41:06 GMT'},
  'RetryAttempts': 0},
 'ETag': '"0b825fd116866b19bc14e190b9683b9c"'}